#掛載雲端

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 載入 AOI瑕疵檢測 資料集

In [ ]:
#將壓縮檔複製到/content
! cp  "/content/drive/MyDrive/AOI/train_images.zip" /content/

In [ ]:
#解壓縮訓練集
! unzip /content/train_images > data_unzip.log

replace train_images/train_00000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
#read csv
import pandas as pd
AOI_data = pd.read_csv('/content/drive/MyDrive/AOI/train.csv')

In [ ]:
#查看訓練集每類別圖片數量並且分類
label = []
for i in range(6):
  #請輸入程式碼
  temp = AOI_data[AOI_data['Label'] == i]
  label.append(temp.reset_index())
  print('第' + str(i) + '類張數:' + str(len(label[i])))


第0類張數:674
第1類張數:492
第2類張數:100
第3類張數:378
第4類張數:240
第5類張數:644


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_images = []
train_label = []
test_images = []
test_label = []
for i in range(6):
  #讀取圖片測試集圖片(這邊的程式碼不可更動)
  images_temp = []
  images_temp1 = []
  label_temp = [i] * 20
  label_temp1 = [i] * 80
  for j in range(20):
    img = cv2.imread('/content/train_images/'+label[i]['ID'][j])
    images_temp.append(cv2.resize(img,(224,224), cv2.INTER_AREA))
  for k in range(20,100):
    img = cv2.imread('/content/train_images/'+label[i]['ID'][k])
    images_temp1.append(cv2.resize(img,(224,224), cv2.INTER_AREA))
  test_images += images_temp
  test_label += label_temp
  train_images += images_temp1
  train_label += label_temp1
train_images = np.array(train_images)
test_images = np.array(test_images)
 
datagen = ImageDataGenerator(
  featurewise_center=True,
  featurewise_std_normalization=True,
  rotation_range=20,
  width_shift_range=0.2,
  height_shift_range=0.2,
  horizontal_flip=True)

datagen.fit(train_images)

H = datagen.flow(train_images, train_label, batch_size=480)

train_images = np.vstack((train_images,H[0][0]))
train_label = np.array(list(train_label) + list(H[0][1]))

In [ ]:
#將 list 轉成 array
import numpy as np
from sklearn.utils import shuffle
x_train = np.array(train_images)
x_test = np.array(test_images)
y_train = np.array(train_label)
y_test = np.array(test_label)

In [ ]:
#將將訓練集進行shuffle
import random
#請輸入程式碼
x_train , y_train = shuffle(x_train, y_train, random_state=random.seed())

# 查看資料的 shape

In [ ]:
print('x_train shape: ', x_train.shape)
print('x_test shape: ', x_test.shape)

x_train shape:  (960, 224, 224, 3)
x_test shape:  (120, 224, 224, 3)


# 查看標籤的 shape

In [ ]:
print('y_train shape:',y_train.shape)
print('y_test shape:', y_test.shape)

y_train shape: (960,)
y_test shape: (120,)


# 查看標籤的內容 

In [ ]:
print(y_train[0:10])

[1 4 4 2 5 3 0 2 4 5]


# 進行 min-max normalization

In [ ]:
x_train_norm = x_train.astype('float32')/255
x_test_norm = x_test.astype('float32')/255

In [ ]:
print(x_train_norm[0][0][0])

[0.75686276 0.75686276 0.75686276]


# 將數字標籤進行 One-hot 編碼

In [ ]:
from tensorflow.keras import utils

# 轉換前
print(y_train[0])

1


In [ ]:
# 進行 One-hot 編碼轉換...
y_train_onehot = utils.to_categorical(y_train, 6)
y_test_onehot = utils.to_categorical(y_test, 6)
# 轉換後
print(y_train_onehot[0])

[0. 1. 0. 0. 0. 0.]


# 建立 CNN 神經網路架構

In [ ]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

import tensorflow.keras.applications as tensorflow_model

In [53]:
#修改全聯階層的輸出類別數
#請輸入程式碼
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=(3, 3), padding='same', input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(6, activation='softmax'))
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_26 (Conv2D)          (None, 224, 224, 128)     3584      
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 112, 112, 128)    0         
 g2D)                                                            
                                                                 
 batch_normalization_8 (Batc  (None, 112, 112, 128)    512       
 hNormalization)                                                 
                                                                 
 dropout_22 (Dropout)        (None, 112, 112, 128)     0         
                                                                 
 conv2d_27 (Conv2D)          (None, 112, 112, 64)      73792     
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 56, 56, 64)     

In [54]:
# 神經網路的訓練配置 #
adam = tf.optimizers.Adam(0.0001)
cnn.compile(optimizer = adam,loss = 'categorical_crossentropy', metrics = ['acc'])

In [55]:
# 進行訓練 #
history = cnn.fit(x=x_train_norm,y=y_train_onehot,batch_size=32,epochs=20,validation_split=0.1)

Epoch 1/20
27/27 [==============================] - 15s 383ms/step - loss: 1.4487 - acc: 0.9132 - val_loss: 1.5747 - val_acc: 0.6771
Epoch 2/20
27/27 [==============================] - 9s 343ms/step - loss: 1.4417 - acc: 0.9201 - val_loss: 1.5601 - val_acc: 0.6875
Epoch 3/20
27/27 [==============================] - 9s 343ms/step - loss: 1.4403 - acc: 0.9178 - val_loss: 1.5190 - val_acc: 0.7500
Epoch 4/20
27/27 [==============================] - 9s 336ms/step - loss: 1.4423 - acc: 0.9062 - val_loss: 1.5905 - val_acc: 0.6250
Epoch 5/20
27/27 [==============================] - 9s 333ms/step - loss: 1.4299 - acc: 0.9271 - val_loss: 1.4707 - val_acc: 0.8646
Epoch 6/20
27/27 [==============================] - 9s 330ms/step - loss: 1.4324 - acc: 0.9329 - val_loss: 1.5726 - val_acc: 0.6562
Epoch 7/20
27/27 [==============================] - 9s 331ms/step - loss: 1.4218 - acc: 0.9537 - val_loss: 1.5088 - val_acc: 0.7708
Epoch 8/20
27/27 [==============================] - 9s 328ms/step - loss: 1

# 使用測試資料評估神經網路

In [56]:
# evaluate
test_loss,test_val = cnn.evaluate(x_test_norm,y_test_onehot)
print('測試資料損失值:', test_loss)
print('測試資料準確度:', test_val)

4/4 [==============================] - 1s 96ms/step - loss: 1.3685 - acc: 0.9750
測試資料損失值: 1.3684967756271362
測試資料準確度: 0.9750000238418579


In [ ]:
# predict
predict_prop = cnn.predict(x_test_norm)
print('第一筆測試資料的預測機率', predict_prop[0])

第一筆測試資料的預測機率 [5.3620076e-01 9.1038318e-03 3.1686342e-01 1.3646436e-01 1.3534890e-03
 1.4220403e-05]


# 將模型轉換成ONNX

In [ ]:
#安裝套件 tf2onnx
!pip install tf-estimator-nightly==2.8.0.dev2021122109 #colab環境缺少此套件 需安裝此套件才能完整安裝tf2onnx
!pip install git+https://github.com/onnx/tensorflow-onnx

In [ ]:
#進行轉換
import tf2onnx
#請輸入程式碼


In [ ]:
#檢查opset版本
#請輸入程式碼


In [ ]:
#由於耐能的量化過程只支持opset ai.onnx 所以移除ai.onnx.ml並儲存
#請輸入程式碼
